In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import linalg
%matplotlib inline

In [135]:
#ソフト閾値関数
def soft_thresholding(x, y):
    return np.sign(x) * max(abs(x) - y, 0)

In [240]:
class Lasso:
    def __init__(self, lambda_, tol=0.0001, max_iter=1000):
        self.lambda_ = lambda_
        self.tol = tol
        self.max_iter = max_iter
        self.w_ = None
        
    def fit(self, X, t):
        #特徴量のcolumns数d、row数nを抽出
        n, d = X.shape
        # w0vec用にwを追加
        self.w_ = np.zeros(d + 1)
        avgl1 = 0.
        # "_"は繰り返し変数を利用しない時に使う
        for _ in range(self.max_iter):
            avgl1_prev = avgl1
            self._update(n, d, X, t)
            avgl1 = np.abs(self.w_).sum() / self.w_.shape[0]
            print("avgl1:{}".format(avgl1))
            # L2ノルムの変化量がtol(0.001)以下になったらループを終了する
            if abs(avgl1 - avgl1_prev) <= self.tol:
                break
    
    def _update(self, n, d, X, t):
        # w0vecを求める　(t - Xw)の合計 / n
        # 例 2変数の場合：(y-w1x+w2)/n = w0 
        self.w_[0] = (t - np.dot(X, self.w_[1:])).sum() / n
        # １列w0vecを追加
        w0vec = np.ones(n) * self.w_[0]
        # 各wについて計算する
        for k in range(d):
            ww = self.w_[1:]
            ww[k] = 0
            q = np.dot(t - w0vec - np.dot(X, ww), X[:, k])
#             print("q:{}".format(q))
            r = np.dot(X[:, k], X[:, k])
#             print("r:{}".format(r))
            self.w_[k + 1] = soft_thresholding(q / r, self.lambda_)
            print("self.w_:{}".format(self.w_[k + 1]))
            
    def predict(self, X):
        if X.ndim == 1:
            X = X.reshape(X.shape[0], 1)
        Xtil = np.c_[np.ones(X.shape[0]), X]
        return np.dot(Xtil, self.w_)

### ラッソ回帰を逐次実行

## 1. 特徴量の生成

In [145]:
#特徴量の生成
np.random.seed(1)
X = np.random.randn(20, 2)
n, d = X.shape

In [146]:
X

array([[ 1.62434536, -0.61175641],
       [-0.52817175, -1.07296862],
       [ 0.86540763, -2.3015387 ],
       [ 1.74481176, -0.7612069 ],
       [ 0.3190391 , -0.24937038],
       [ 1.46210794, -2.06014071],
       [-0.3224172 , -0.38405435],
       [ 1.13376944, -1.09989127],
       [-0.17242821, -0.87785842],
       [ 0.04221375,  0.58281521],
       [-1.10061918,  1.14472371],
       [ 0.90159072,  0.50249434],
       [ 0.90085595, -0.68372786],
       [-0.12289023, -0.93576943],
       [-0.26788808,  0.53035547],
       [-0.69166075, -0.39675353],
       [-0.6871727 , -0.84520564],
       [-0.67124613, -0.0126646 ],
       [-1.11731035,  0.2344157 ],
       [ 1.65980218,  0.74204416]])

In [147]:
# 特徴量の行数
n

20

In [148]:
# 特徴量の数（列数）
d

2

In [207]:
# 出力データ
t = np.random.rand(20)
t

array([0.14352857, 0.30130892, 0.75586753, 0.34292079, 0.60364942,
       0.62139259, 0.29861366, 0.46090939, 0.68479916, 0.77365904,
       0.64721797, 0.38573907, 0.0315746 , 0.90100791, 0.00937518,
       0.10238995, 0.78327418, 0.06089223, 0.88579634, 0.89811097])

In [205]:
X.shape

(20, 2)

In [206]:
t.shape

(20,)

## 2. 定数項の計算

In [150]:
# 定数項を追加する
w_ = np.zeros(d + 1)
w_

array([0., 0., 0.])

In [208]:
# 定数項の値を計算する
w_[0] = (t - np.dot(X, w_[1:])).sum() / n
w_

array([0.48460137, 0.        , 0.        ])

## 2. w1の計算

In [209]:
# １列w0vecを追加
w0vec = np.ones(n) * w_[0]
w0vec

array([0.48460137, 0.48460137, 0.48460137, 0.48460137, 0.48460137,
       0.48460137, 0.48460137, 0.48460137, 0.48460137, 0.48460137,
       0.48460137, 0.48460137, 0.48460137, 0.48460137, 0.48460137,
       0.48460137, 0.48460137, 0.48460137, 0.48460137, 0.48460137])

In [210]:
# 定数項以降のwを抽出
ww = w_[1:]
ww

array([0., 0.])

In [211]:
# w1を0とおく
k = 0
ww[k] = 0

In [212]:
# w1の値を各行について計算
# 式05-09の分子の内側((y - wo - w1) * w1)の計算
q = np.dot(t - w0vec - np.dot(X, ww), X[:, k])
q

-0.23935912345858779

In [213]:
# 特徴量　行は全て抽出（:）, 列はkのみ抽出していることを確認　今回はk=0
X[:, k]

array([ 1.62434536, -0.52817175,  0.86540763,  1.74481176,  0.3190391 ,
        1.46210794, -0.3224172 ,  1.13376944, -0.17242821,  0.04221375,
       -1.10061918,  0.90159072,  0.90085595, -0.12289023, -0.26788808,
       -0.69166075, -0.6871727 , -0.67124613, -1.11731035,  1.65980218])

In [214]:
# 分母の計算
r = np.dot(X[:, k], X[:, k])
r

18.698340609659205

In [215]:
# lambdaの設定
lambda_ = 0.1

### ソフト閾値関数の計算

In [216]:
q / r

-0.012801089062145945

In [217]:
abs(q /r) - lambda_

-0.08719891093785406

In [223]:
max(abs(q/r) - lambda_, 0)

0

In [219]:
np.sign(r)

1.0

In [232]:
# ソフト閾値関数を適用
w_[k+1] = np.sign(q/r) + max(abs(q/r) - lambda_, 0)
w_

array([ 0.48460137, -1.        ,  0.        ])

In [233]:
#
np.abs(w_).sum()

1.4846013741928348

In [236]:
w_.shape[0]

3

In [237]:
np.abs(w_).sum() / w_.shape[0]

0.4948671247309449

In [117]:
import csv

In [197]:
# データ読み込み
Xy = []
with open("winequality-red.csv") as fp:
    for row in csv.reader(fp, delimiter=";"):
        Xy.append(row)
Xy = np.array(Xy[1:], dtype=np.float64)

np.random.seed(0)
np.random.shuffle(Xy)
train_X = Xy[:-1000, :-1]
train_y = Xy[:-1000, -1]
test_X = Xy[-1000:, :-1]
test_y = Xy[-1000:, -1]

In [201]:
train_X.shape

(599, 11)

In [200]:
train_y.shape

(599,)

In [241]:
#ハイパーパラメータを変えながら学習させて結果表示
for lambda_ in [1., 0.1, 0.01]:
    model = Lasso(lambda_)
    model.fit(train_X, train_y)
    y = model.predict(test_X)
    print(" --- lambda = {} ---".format(lambda_))
    print("coefficients:")
    print(model.w_)
    mse = ((y - test_y)**2).mean()
    print("MSE: {:.3f}".format(mse))

self.w_:0.0
self.w_:-0.0
self.w_:0.0
self.w_:0.0
self.w_:-0.0
self.w_:0.0
self.w_:-0.0
self.w_:-0.0
self.w_:-0.0
self.w_:0.0
self.w_:0.0
avgl1:0.4653589315525877
self.w_:0.0
self.w_:-0.0
self.w_:0.0
self.w_:0.0
self.w_:-0.0
self.w_:0.0
self.w_:-0.0
self.w_:-0.0
self.w_:-0.0
self.w_:0.0
self.w_:0.0
avgl1:0.4653589315525877
 --- lambda = 1.0 ---
coefficients:
[ 5.58430718  0.         -0.          0.          0.         -0.
  0.         -0.         -0.         -0.          0.          0.        ]
MSE: 0.691
self.w_:0.0
self.w_:-0.03660204370566805
self.w_:0.15474403049651828
self.w_:-0.0
self.w_:-0.5406510989457188
self.w_:0.0
self.w_:-0.0
self.w_:0.0
self.w_:0.0
self.w_:0.0
self.w_:0.0
avgl1:0.5263586959815797
self.w_:0.0
self.w_:-0.05980105008734404
self.w_:0.24359213465817411
self.w_:-0.0
self.w_:-0.8945484143822595
self.w_:0.0
self.w_:-0.0
self.w_:0.0
self.w_:0.0
self.w_:0.0
self.w_:0.0
avgl1:0.5671067536588954
self.w_:0.0
self.w_:-0.07504552753815291
self.w_:0.2944177801167962
self.w